## 0. Questions

 - What is Linear Regression?
 - Which methods are used for optimization?
     - When is it suitable to use quantitative approach (gradient descent) instead of analytical solution?
     - What is analytical solution? How to derive the formula?
     - What is gradient descent and how can it be used to optimize parameters?
 - Which metrics are used to evaluate the performance of a regression model?
     - Describe MAE, MSE, RMSE, MAPE, MedAE
 - What is $R^2$ (coefficient of determination)?
     - In which case $R^2$ can be lower than 0?
 - What is regularization? And why is it needed?
     - What is Lasso?
     - What is Ridge?
     - What is the difference between Lasso and Ridge?
     - What is Elastic Net?
     - How to find the regularization parameter in LASSO or Ridge or Elastic Net?
     - In what situations LASSO performs better than Ridge?



## 1. Import packages

First, let's import needed modules and set random seed (we'll use it if needed).

In [38]:
import os
import sys
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split

from linear_regression_implementations import SciPyLinearRegressionOLS, LinearRegressionOLS, SciPyLinearRegressionNNOLS

SEED = 42

## 2. Data preparation

### 2.1 Loading california housing dataset

In [6]:
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

In [7]:
X.shape

(20640, 8)

### 2.2 Splitting the data

Splitting data into training and testing dataset. 10% for test set will be enough, because we have enough data

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

### 2.3 Scaling the data

Standardize features by removing the mean and scaling to unit variance

In [27]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.scaler import StandardScaler

In [34]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 3. Predictive modelling

### 3.1 Analytical solution

First solution will use the analytical solution to OLS to get the weights/coefficients: $\hat{\beta} = (X^TX)^{-1}(X^TY)$

#### 3.1.1 Wrapper for analytical solution

Before fitting we should include first column of 1's to `X_train_scaled` matrix and check the $det(X^TX)$ - it should not be equal to 0 - otherwise we would not be able to invert the matrix $(X^TX)$

In [39]:
ols_reg = LinearRegressionOLS()
ols_reg.fit(X_train_scaled, y_train)
print(f"The coefficients are {ols_reg.coef_}")
print(f"The intercept is {ols_reg.intercept_}")

The coefficients are [ 0.84256988  0.11978692 -0.27791665  0.32476338 -0.00115151 -0.03999763
 -0.89935429 -0.87044066]
The intercept is 2.0729521328596032


#### 3.1.2 Sklearn's implementation

In [40]:
sklearn_ols_reg = LinearRegression(positive=False)
sklearn_ols_reg.fit(X_train_scaled, y_train)
print(f"The coefficients are {sklearn_ols_reg.coef_}")
print(f"The intercept is {sklearn_ols_reg.intercept_}")

The coefficients are [ 0.84256988  0.11978692 -0.27791665  0.32476338 -0.00115151 -0.03999763
 -0.89935429 -0.87044066]
The intercept is 2.072952132859605


#### 3.1.3 Wrapper for `scipy.linalg.lstsq` method

As we can see the results are the same, because sklearn is using <code>scipy.linalg.lstsq</code> method under the hood. Which, also, wrapped by me below:

In [41]:
scipy_ols_reg = SciPyLinearRegressionOLS()
scipy_ols_reg.fit(X_train_scaled, y_train)
print(f"The coefficients are {scipy_ols_reg.coef_}")
print(f"The intercept is {scipy_ols_reg.intercept_}")

The coefficients are [ 0.84256988  0.11978692 -0.27791665  0.32476338 -0.00115151 -0.03999763
 -0.89935429 -0.87044066]
The intercept is 2.0729521328596046


The results are expectedly the same


### 3.2 Non-negative least-squares

We would want to constrain the coefficients to non-negative values whenever a negative value makes no physical sense, say because it represents the intensity of a pixel, or the price of an object, or a frequency count, or a chemical concentration, etc.

#### 3.2.1 Sklearn's implementation for non-negative least squares

When <code>positive</code> set to <code>True</code>, during initialization of <code>sklearn.linear_model.LinearRegression</code> object, it will force the coefficients to be positive.

We can see that the solution looks completely different - all the coefficients are either zero or positive.

In [42]:
sklearn_ols_reg_pos = LinearRegression(positive=True)
sklearn_ols_reg_pos.fit(X_train_scaled, y_train)
print(f"The coefficients are {sklearn_ols_reg_pos.coef_}")
print(f"The intercept is {sklearn_ols_reg_pos.intercept_}")

The coefficients are [0.82392238 0.23238543 0.         0.0242489  0.0405112  0.
 0.         0.        ]
The intercept is 2.0729521328596037


#### 3.2.2 Wrapper for `scipy.optimize.nnls` method

Which is naturally equal to the results from the wrapped method

In [43]:
scipy_nn_ols_reg = SciPyLinearRegressionNNOLS()
scipy_nn_ols_reg.fit(X_train_scaled, y_train)
print(f"The coefficients are {scipy_nn_ols_reg.coef_}")
print(f"The intercept is {scipy_nn_ols_reg.intercept_}")

The coefficients are [0.82392238 0.23238543 0.         0.0242489  0.0405112  0.
 0.         0.        ]
The intercept is 2.0729521328596245


### 3.3 Gradient Descent solution

Gradient Descent (GD) or Stochastic Gradient Descent (SGD) solution is needed for regression problems with a large number of training samples and features (> 100 000)

In [44]:
sklearn_sgd_reg = SGDRegressor()
sklearn_sgd_reg.fit(X_train_scaled, y_train)
print(f"The coefficients are {sklearn_sgd_reg.coef_}")
print(f"The intercept is {sklearn_sgd_reg.intercept_}")

The coefficients are [ 0.45362911  0.11823577  0.35700974 -0.45554855  0.03781292 -2.67187916
 -1.44594935 -1.35306023]
The intercept is [2.0188528]


### 4. Metrics

#### 4.1 $R^2$

The $R^2$ is the coefficient of determination which is defined as $R^2 = 1 -\frac{\sum\limits _{i = 1} ^N (y_i - \hat y_i)}{\sum\limits _{i = 1} ^N (y_i - \bar y_i)} = 1 - \frac{SS_{fit}}{SS_{mean}}$. <br>
It shows how much the fit of the chosen model is better than a fit of a horizontal straight line (mean value for target $y$). <br>
The best possible score is 1.0, and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of $y$, disregarding the input features, would get a $R^2$ score of 0.0. <br>
$R^2$ can have a negative value without violating any rules of math. $R^2$ is negative only when the chosen model does not follow the trend of the data, so fits worse than a $y$-mean line.

In [11]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
print(f"R2 on the train data is equal to {lr.score(X_train_scaled, y_train).round(2)}")
print(f"R2 on the test data is equal to {lr.score(X_test_scaled, y_test).round(2)}")


R2 on the train data is equal to 0.61
R2 on the test data is equal to 0.6


# TODO:
 - the idea of regularization and why is it needed
 - using gradient descent instead of using formula (analytical solution)
 - comparing results of SGDRegressor with results of ridge regression
 - in which situations using gradient descent is more advisable
 - Linear regression metrics
      - R2, adjusted R2 and statistical significance
 - Non-linear methods for regression (decision trees and random forest regressors)
 - Write down questions about regression